In [ ]:
# necessary for DenodoConnection
#%pip install psycopg2-binary
#%pip install shap

In [2]:
# import packages
import os
import sys

import pandas as pd, numpy as np
import json
import argparse
import os

import pickle
import warnings

# need to add dme_sagemaker to path to load in libraries
sys.path.append("/root/dme_sagemaker/dme_sagemaker")

from libs.denodo.denodo_connection import DenodoConnection
import libs.performance_lib.performance_sql_recipes as performance_sql_recipes
from libs.helper.parameters_helper import ParametersHelper

from libs.performance_lib import predictive_advancement_lib
from libs.performance_lib import performance_validation_lib

import matplotlib.pyplot as plt
import boto3

ENVIRONMENT name from config_vars: uat
DME_PROJECT name from config_vars: None


In [3]:
DKU_DST_ap_data_sector = 'CORN_BRAZIL_SUMMER'
DKU_DST_analysis_type = 'GenoPred'
input_years = ['2020','2021','2022']
output_year = '2022'
bucket = 'us.com.syngenta.ap.nonprod'

In [4]:
# when testing, we read directly from S3, bucket is provided as a separate input to function, don't include in input args. 
input_args = [
    '--s3_input_pred_adv_data_folder', 'uat/dme/performance/compute_pred_adv_data_collected/data/'
]

parser = argparse.ArgumentParser(description='app inputs and outputs')
parser.add_argument('--s3_input_pred_adv_data_folder', type=str,
                    help='s3 input pred_adv_data folder', required=True)
args = parser.parse_args(input_args)

In [5]:
potential_fnames = predictive_advancement_lib.load_potential_fnames(prefix='new_')

df_input_piv = predictive_advancement_lib.load_and_preprocess_all_inputs_ml(
    args,
    DKU_DST_ap_data_sector,
    DKU_DST_analysis_type,
    potential_fnames,
    years_to_load=input_years,
    read_from_s3=1,
    bucket=bucket
)

/opt/conda/lib/python3.7/site-packages/boto3/compat.py:82: PythonDeprecationWarning: Boto3 will no longer support Python 3.7 starting December 13, 2023. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.8 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  # Remove the CWD from sys.path while we load stuff.


(48021, 47)
(118128, 9)
(22488, 54)
(4246, 21)
(11, 4)
(77136, 9)
(20945, 53)
(7579, 21)
(187155, 25)
(35, 4)
(61583, 9)


In [13]:
df_input_piv[df_input_piv['material_type_simple'] == 'parent'][['entry_identifier','current_stage','analysis_year']].groupby(by=['current_stage','analysis_year']).count()

entry_identifier
current_stage analysis_year                  
1.0           2020                      18960
              2021                       4319
              2022                       3584
3.0           2020                       1747
              2021                        885
              2022                       2220
4.0           2020                       1835
              2021                       3201
              2022                       2368
5.0           2020                        436
              2021                       1073
              2022                        703
6.0           2020                        148
              2021                        144
              2022                        159
7.0           2020                        140
              2021                        288
              2022                        160

In [7]:
for col in df_input_piv.columns:
    print(col, df_input_piv[col][df_input_piv['material_type_simple'] != 'entry'].count())

ap_data_sector 42640
analysis_year 42640
entry_identifier 42640
material_type_simple 42640
decision_group 42640
chkfl_ERHTN 14347
chkfl_GMSTP 41553
chkfl_GWTPN 41473
chkfl_HAVPN 2987
chkfl_LRTLR 37253
chkfl_PLHTN 14391
chkfl_SDSWN 19258
chkfl_STD_N 6227
chkfl_STD_P 2906
chkfl_STKLR 37304
chkfl_TWSMN 22617
chkfl_YGHMN 41490
chkfl_YGMRN 41490
chkfl_YGSMN 41490
result_ERHTN 14347
result_GMSTP 41553
result_GWTPN 41473
result_HAVPN 2987
result_LRTLR 37253
result_PLHTN 14391
result_SDSWN 19258
result_STD_N 6227
result_STD_P 2906
result_STKLR 37304
result_TWSMN 22617
result_YGHMN 41490
result_YGMRN 41490
result_YGSMN 41490
result_diff_ERHTN 14347
result_diff_GMSTP 41553
result_diff_GWTPN 41473
result_diff_HAVPN 2987
result_diff_LRTLR 37253
result_diff_PLHTN 14391
result_diff_SDSWN 19258
result_diff_STD_N 6227
result_diff_STD_P 2906
result_diff_STKLR 37304
result_diff_TWSMN 22617
result_diff_YGHMN 41490
result_diff_YGMRN 41490
result_diff_YGSMN 41490
current_stage 42370
next_stage 42370
prev_s

In [ ]:
result_diff_ERHTN 14347
result_diff_GMSTP 41553
result_diff_GWTPN 41473
result_diff_HAVPN 2987
result_diff_LRTLR 37253
result_diff_PLHTN 14391
result_diff_SDSWN 19258
result_diff_STD_N 6227
result_diff_STD_P 2906
result_diff_STKLR 37304
result_diff_TWSMN 22617
result_diff_YGHMN 41490
result_diff_YGMRN 41490
result_diff_YGSMN 41490

In [14]:
df_piv = df_input_piv[(df_input_piv['material_type'] != 'entry') &
                          (df_input_piv['current_stage'] <= 3) &
                          (df_input_piv['current_stage'] >= 1)]

In [17]:
traits = ['ERHTN','GMSTP','GWTPN','HAVPN','LRTLR','PLHTN','SDSWN','STD_N','STD_P','STKLR','TWSMN','YGHMN','YGSMN']

df_piv[['was_adv'] + ['result_diff_' + trait for trait in traits]].corr()

,was_adv,result_diff_ERHTN,result_diff_GMSTP,result_diff_GWTPN,result_diff_HAVPN,result_diff_LRTLR,result_diff_PLHTN,result_diff_SDSWN,result_diff_STD_N,result_diff_STD_P,result_diff_STKLR,result_diff_TWSMN,result_diff_YGHMN,result_diff_YGSMN
was_adv,1.000000,0.054042,0.017101,0.122570,NaN,0.000253,0.036219,0.013600,-0.036137,0.015588,-0.008210,0.012219,0.120492,0.122431
result_diff_ERHTN,0.054042,1.000000,0.092357,0.323369,NaN,0.050820,0.788181,0.073581,0.114654,-0.048135,-0.014934,0.101932,0.323071,0.313839
result_diff_GMSTP,0.017101,0.092357,1.000000,0.259468,NaN,-0.017504,0.044174,-0.106912,-0.005397,-0.032541,-0.059343,0.127563,0.262127,0.020197
result_diff_GWTPN,0.122570,0.323369,0.259468,1.000000,NaN,-0.049465,0.328490,0.079331,-0.006644,NaN,-0.128214,0.041983,0.997167,0.964965
result_diff_HAVPN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
result_diff_LRTLR,0.000253,0.050820,-0.017504,-0.049465,NaN,1.000000,0.042470,-0.002888,NaN,NaN,0.068337,0.043682,-0.051271,-0.050369
result_diff_PLHTN,0.036219,0.788181,0.044174,0.328490,NaN,0.042470,1.000000,0.086000,0.219181,-0.053617,-0.000275,0.081412,0.327603,0.329032
result_diff_SDSWN,0.013600,0.073581,-0.106912,0.079331,NaN,-0.002888,0.086000,1.000000,NaN,NaN,0.004726,0.045156,0.079308,0.107907
result_diff_STD_N,-0.036137,0.114654,-0.005397,-0.006644,NaN,NaN,0.219181,NaN,1.000000,0.962510,NaN,NaN,0.052835,0.058242
result_diff_STD_P,0.015588,-0.048135,-0.032541,NaN,NaN,NaN,-0.053617,NaN,0.962510,1.000000,NaN,NaN,0.006334,0.007290


In [ ]:
def process_harvt(row_in):
    if isinstance(row_in,str):
        return 'drop' in row_in.lower()
    else:
        return False
    
# convert each entry in input years to an integer since they come in as str.
if isinstance(input_years, list):
    input_years_as_int = [int(val) for val in input_years]
else:
    input_years_as_int = [int(input_years)]

#### FILTER TO DESIRED DATA for training
# filter materials: want stages 1-4, material type is parent for corn
# want stages 2-5, material type is entry for soy
if DKU_DST_ap_data_sector == "CORN_NA_SUMMER":
    df_piv = df_input_piv[(df_input_piv['material_type'] != 'entry') &
                          (df_input_piv['current_stage'] <= 4) &
                          (df_input_piv['current_stage'] >= 2)]

    # drop edge decision groups when training
    df_piv = df_piv[(df_piv['decision_group_rm'] > 80) & (df_piv['decision_group_rm'] < 120)]

    # drop any materials that had a selection_remark == drop for training
    df_piv = df_piv[df_piv['selection_remark'] != 9]

elif DKU_DST_ap_data_sector == "CORN_BRAZIL_SUMMER":
    df_piv = df_input_piv[(df_input_piv['material_type'] != 'entry') &
                          (df_input_piv['current_stage'] <= 3) &
                          (df_input_piv['current_stage'] >= 1)]
    
    numeric_raw_traits = []
    numeric_diff_traits = []

    
elif DKU_DST_ap_data_sector == 'SOY_NA_SUMMER':
    df_input_piv['harvt_drop'] = df_input_piv['harvt'].apply(process_harvt)
    df_piv = df_input_piv[(df_input_piv['material_type'] == 'entry') &
                          (df_input_piv['current_stage'] >= 3) &
                          (df_input_piv['current_stage'] <= 3) &
                          (df_input_piv['harvt_drop']==False)]

    #### preprocess text traits for soy
    extra_traits = []
    #numeric_raw_traits = ['PLHTN', 'IC__N', 'PRTNP', 'OIL_P', 'SDS_R']
    #numeric_diff_traits = ['E3_R', 'EMRGR', 'MRTYN',
    #                       'SBSSN', 'YGSMN', 'HLDGR', 'DIC_R', 'FL_CR', 'GLU_R', 'HVAPR',
    #                       'MI__R', 'PD_CR', 'RUR_R']
    numeric_raw_traits = []
    numeric_diff_traits = ['MRTYN','PLHTN','SDS_R','EPA_R',
                           'YGSMN', 'FL_CR', 'HVAPR',
                           'PD_CR','GMSTP','PB_CR']
    
    text_traits = []#['bp_t', 'bsr_t', 'cls_t', 'cn3_t', 'dic_t',
                   #'dpm_t', 'e1_t', 'e3_t', 'fels_t', 'fl_ct', 'hilct', 'll55_t',
                   #'met_t', 'mi__t', 'pb_ct', 'rr2_t',
                   #'stmtt', 'sts_t','rps_t'] #pd_ct'

    text_suffix = {'bsr_t': 'Rbs1', 'fels_t': 'Rcs3'}
    text_prefix = {'fels_t': '', 'bp_t': '', 'bsr_t': '', 'cls_t': '', 'dic_t': '', 'dpm_t': '',
                   'e1_t': '', 'e3_t': '', 'fl_ct': '', 'll55_t': '', 'met_t': '',
                   'mi__t': '', 'pb_ct': '', 'pd_ct': '', 'rr2_t': '', 'stmtt': '', 'sts_t': ''}

    clean_text = {'cls_t': '_'}

elif DKU_DST_ap_data_sector == 'SOY_BRAZIL_SUMMER':
    df_input_piv['harvt_drop'] = df_input_piv['harvt'].apply(process_harvt)
    df_piv = df_input_piv[(df_input_piv['material_type'] == 'entry') &
                          (df_input_piv['current_stage'] >= 5) &
                          (df_input_piv['current_stage'] <= 6) &
                          (df_input_piv['harvt_drop'] == False)]


    #### preprocess text traits for soy
    extra_traits = []
    numeric_raw_traits = []
    numeric_diff_traits = ['YGSMN','HLDGR','MRTYN']
    
elif DKU_DST_ap_data_sector == 'SOY_LAS_SUMMER':
    df_input_piv['harvt_drop'] = df_input_piv['harvt'].apply(process_harvt)
    df_piv = df_input_piv[(df_input_piv['material_type'] == 'entry') &
                          (df_input_piv['current_stage'] >= 3) &
                          (df_input_piv['current_stage'] <= 5) &
                          (df_input_piv['harvt_drop'] == False)]


    #### preprocess text traits for soy
    extra_traits = []
    numeric_raw_traits = []
    numeric_diff_traits = ['YGSMN']

In [ ]:
### train model.
grouping_cols = predictive_advancement_lib.get_grouping_cols_per_sector(analysis_sector=DKU_DST_ap_data_sector)
df_piv_grouped = df_piv.groupby(by=grouping_cols)

In [ ]:
for index, df_use in df_piv_grouped:  # build model per unique combination of grouping_cols; stage currently
    # get model parameters for this sector and stage
    args = []
    if DKU_DST_ap_data_sector == "SOY_NA_SUMMER":
        args = [extra_traits, numeric_raw_traits, numeric_diff_traits, text_traits]

    # model parameters
    # extract inputs from args
    # get input and output column names
    mdl_in_cols = []
    mdl_in_cols.extend(['result_diff_' + trait for trait in numeric_diff_traits])
    mdl_in_cols.extend(['result_' + trait for trait in numeric_raw_traits])

    cols_to_norm = []
    mdl_out_col = 'was_adv'

    # get parameters for xgboost models
    xgb_params_class = predictive_advancement_lib.XGBoostModelParameters(
        max_depth=5,
        reg_lambda=100,
        subsample=0.2,
        learning_rate=0.005,
        n_estimators=500
    )

    # make sure each column in mdl_in col is also in the dataframe
    mdl_in_cols_use = mdl_in_cols.copy()
    for col in mdl_in_cols:
        if col not in df_use.columns:
            mdl_in_cols_use.remove(col)
    mdl_in_cols = mdl_in_cols_use
        
    # if the output var is not a boolean, this causes problems. check
    if not isinstance(df_use[mdl_out_col].dtype, bool):
        df_use[mdl_out_col] = df_use[mdl_out_col].astype(bool)

    if DKU_DST_ap_data_sector == 'CORN_NA_SUMMER':
        preproc_class = predictive_advancement_lib.PredAdvPreprocessor(
            preprocess_steps=['subtract corr', 'process rm', 'aggregate cols'],
            corr_traits=[[['PLHTN', 'ERHTN', 'GMSTP'], ['YGSMN']], [['ERHTN'], ['PLHTN']]],
            corr_score_col='prediction',
            to_agg=[['stderr_YGSMN', 'stderr_ERHTN', 'stderr_GRSNP', 'stderr_TWSMN']],
            agg_names=['prediction_reliability'], agg_funcs=[np.nanmean])

        mdl_class = predictive_advancement_lib.PredAdvMdl(
            params=xgb_params_class.params,
            in_cols=mdl_in_cols,
            out_col=mdl_out_col,
            weight_by_col=False,
            weight_col='was_adv_next',
            weight=25,
            perform_post_punishing=True,
            post_punishing_data={'prediction_GMSTP': (0.75, 0.05)}
        )

    elif DKU_DST_ap_data_sector == 'SOY_NA_SUMMER':
        preproc_class = predictive_advancement_lib.PredAdvPreprocessor(
            preprocess_steps=['subtract corr'],
            corr_traits=[[['MRTYN'],['YGSMN']]],
            corr_score_col='result',
            text_traits=text_traits,
            text_suff=text_suffix,
            text_pref=text_prefix,
            clean_text=clean_text
        )

        mdl_class = predictive_advancement_lib.PredAdvMdl(
            params=xgb_params_class.params,
            in_cols=mdl_in_cols,
            out_col=mdl_out_col,
            weight_by_col=False
        )
    elif DKU_DST_ap_data_sector == 'SOY_BRAZIL_SUMMER' or DKU_DST_ap_data_sector == "SOY_LAS_SUMMER":
        preproc_class = predictive_advancement_lib.PredAdvPreprocessor(
            preprocess_steps=[],
        )

        mdl_class = predictive_advancement_lib.PredAdvMdl(
            params=xgb_params_class.params,
            in_cols=mdl_in_cols,
            out_col=mdl_out_col,
            weight_by_col=False
        ) 

    train_mask = np.any(
        df_use['analysis_year'].values.reshape(-1, 1) == np.array(input_years_as_int).reshape(1, -1), axis=1
    )
    df_tr_proc = preproc_class.train_preprocessing(df_use.iloc[train_mask == True])

    # update mdl in cols based on preprocessing steps
    if 'process rm' in preproc_class.preprocess_steps:
        mdl_class.in_cols.append('rm_estimate_proc')
        # if there are dummy vars added to df_use_proc due to processing text traits, add to mdl in cols
    if 'process text traits' in preproc_class.preprocess_steps:
        new_cols = list(set(df_tr_proc.columns).difference(set(df_use.columns)))
        if 'process rm' in preproc_class.preprocess_steps:
            mdl_class.in_cols.remove('rm_estimate_proc')
        mdl_class.in_cols.extend(new_cols)
    if 'get qualifiers' in preproc_class.preprocess_steps:
        cols_to_append = df_tr_proc.columns[['qualifiers_' in col for col in df_tr_proc.columns]]
        mdl_in_cols.extend(cols_to_append)

    print(df_tr_proc.shape)
    mdl_class.train_mdl(df_tr_proc);
    df_tr_proc['recommendation_score'] = mdl_class.predict_proba(df_tr_proc)
    df_input_stats = performance_validation_lib.compute_input_statistics(
        df_in=df_tr_proc,
        in_cols=mdl_class.in_cols
    )
    # get predictions, print metrics on training dataset (not representative of performance).
    print(index,df_tr_proc.shape,mdl_class.score(df_tr_proc, df_tr_proc[mdl_out_col].values))

In [ ]:
import xgboost
xgboost.__version__

In [ ]:
import pickle
with open('/root/dme_sagemaker/dme_sagemaker/libs/performance_lib/SOY_PYT_mdl_xgb_v162.pkl','wb') as f:
    pickle.dump(mdl_class,f)

In [ ]:
#shap code for new class method
# get and plot shapley values after training a model
#import shap
do_shap = 0
if do_shap == 1
    x = df_tr_proc[mdl_class.in_cols].values.astype(float)
    shap_input = shap.utils.sample(x, np.minimum(x.shape[0],1000))
    explainer = shap.Explainer(mdl_class.mdl.predict_proba, shap_input)
    shap_values = explainer(shap_input)
    shap_values = shap_values[:,:,1]

    shap.summary_plot(shap_values, feature_names=mdl_class.in_cols,\
                     plot_type='bar')

    shap.summary_plot(shap_values, feature_names=mdl_class.in_cols,\
                     plot_type='dot')

#for idx in range(len(mdl_class.in_cols)):
#    notnan_mask = np.isnan(shap_input[:,idx])==False
#    if np.sum(notnan_mask) > 10:
#        shap.plots.partial_dependence(idx,mdl_class.mdl.predict,shap_input[notnan_mask,:],ice=False,feature_names=mdl_class.in_cols)

# some useful shapley/interpolation related code
#plt.plot(shap_values[:,0].data, shap_values[:,0].values,'.')
#shap.plots.scatter(shap_values[:,5])

In [ ]:
plt.hist(df_tr_proc['result_diff_MRTYN'])
plt.show()